In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display, Math, Latex
from matplotlib import pyplot as plt
import re
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [2]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor

# 1. ЗАГРУЗКА ДАННЫХ
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train = train[train['price_p05'] > 0]
train.shape

train['dt'] = pd.to_datetime(train['dt'])
test['dt'] = pd.to_datetime(test['dt'])

# 2. ДОБАВЛЕНИЕ ЦИКЛИЧЕСКИХ ПРИЗНАКОВ
def add_cyclical_features(df):
    df['dow_sin'] = np.sin(2 * np.pi * df['dow'] / 7)
    df['dow_cos'] = np.cos(2 * np.pi * df['dow'] / 7)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df['day_sin'] = np.sin(2 * np.pi * df['day_of_month'] / 31)
    df['day_cos'] = np.cos(2 * np.pi * df['day_of_month'] / 31)
    return df

train = add_cyclical_features(train)
test = add_cyclical_features(test)

# 3. ОПРЕДЕЛЕНИЕ ПРИЗНАКОВ И КАТЕГОРИЙ
# Важно: CatBoost любит, когда категории передаются как int или string
cat_features = [
    'management_group_id', 'first_category_id', 'second_category_id', 
    'third_category_id', 'product_id', 
    # 'holiday_flag', 
    'activity_flag'
]

features = [
    'n_stores', 'precpt', 'avg_temperature', 'avg_humidity', 
    'avg_wind_level', 'week_of_year',
    'dow_sin', 'dow_cos', 
    'month_sin', 'month_cos', 'day_sin', 'day_cos'
] + cat_features

# Приводим категориальные признаки к типу int, чтобы CatBoost не ругался
for col in cat_features:
    train[col] = train[col].astype(int)
    test[col] = test[col].astype(int)

# 4. ВАЛИДАЦИЯ (Разбиение 80/20 внутри каждого дня)
train_parts = []
val_parts = []

# Группируем по дате и делим каждую группу
for date, group in train.groupby('dt'):
    # Перемешиваем данные внутри дня (опционально, но желательно)
    group = group.sample(frac=1, random_state=42).reset_index(drop=True)
    
    # Вычисляем индекс разделения
    split_idx = int(len(group) * 0.8)
    
    train_parts.append(group.iloc[:split_idx])
    val_parts.append(group.iloc[split_idx:])

# Собираем все обратно в DataFrame
train_part = pd.concat(train_parts).reset_index(drop=True)
val_part = pd.concat(val_parts).reset_index(drop=True)

print(f"Размер обучающей выборки: {train_part.shape}")
print(f"Размер валидационной выборки: {val_part.shape}")

# Далее идет расчет IoU и обучение моделей (как в твоем коде)

def calculate_iou(lower_true, upper_true, lower_pred, upper_pred, epsilon=1e-6):
    intersection = np.maximum(0, np.minimum(upper_true, upper_pred) - np.maximum(lower_true, lower_pred))
    union = (upper_true - lower_true + epsilon) + (upper_pred - lower_pred + epsilon) - intersection
    return np.mean(intersection / union)

# Изменяем параметры CatBoost
cb_params = {
    'iterations': 2000,          # Увеличим число итераций, т.к. early stopping сам остановит обучение
    'learning_rate': 0.03,
    'depth': 8,
    'loss_function': 'MAE',      # Или 'Quantile:alpha=0.05' / 'Quantile:alpha=0.95'
    'random_seed': 42,
    'verbose': 100,
    'early_stopping_rounds': 500, # Остановиться, если метрика не улучшается 100 итераций
    'thread_count': -1
}

print("Обучение моделей с использованием валидации и Early Stopping...")

# Обучаем модель для нижней границы
model_low_val = CatBoostRegressor(**cb_params)
model_low_val.fit(
    train_part[features], train_part['price_p05'],
    eval_set=(val_part[features], val_part['price_p05']), # Добавляем валидационный сет
    cat_features=cat_features,
    use_best_model=True # Использовать веса лучшей итерации
)

# Обучаем модель для верхней границы
model_high_val = CatBoostRegressor(**cb_params)
model_high_val.fit(
    train_part[features], train_part['price_p95'],
    eval_set=(val_part[features], val_part['price_p95']), # Добавляем валидационный сет
    cat_features=cat_features,
    use_best_model=True
)

# Теперь смотрим сколько итераций реально понадобилось
print(f"Лучшая итерация low: {model_low_val.get_best_iteration()}")
print(f"Лучшая итерация high: {model_high_val.get_best_iteration()}")

p_low_val = model_low_val.predict(val_part[features])
p_high_val = model_high_val.predict(val_part[features])
val_score = calculate_iou(val_part['price_p05'], val_part['price_p95'], p_low_val, p_high_val)
print(f"Валидационный IoU (CatBoost): {val_score:.4f}")

# ФИНАЛЬНОЕ ОБУЧЕНИЕ на полном датасете
print("Обучение финальной модели на полном датасете...")

# Создаем копии параметров, но фиксируем количество итераций из валидации
params_low_final = cb_params.copy()
params_low_final['iterations'] = model_low_val.get_best_iteration() + 1
params_low_final.pop('early_stopping_rounds', None) # Удаляем за ненадобностью

params_high_final = cb_params.copy()
params_high_final['iterations'] = model_high_val.get_best_iteration() + 1
params_high_final.pop('early_stopping_rounds', None)

final_model_low = CatBoostRegressor(**params_low_final)
final_model_high = CatBoostRegressor(**params_high_final)

final_model_low.fit(train[features], train['price_p05'], cat_features=cat_features)
final_model_high.fit(train[features], train['price_p95'], cat_features=cat_features)

# 6. ПРЕДСКАЗАНИЕ ДЛЯ TEST.CSV
print("Создание предсказаний...")
test['price_p05'] = final_model_low.predict(test[features])
test['price_p95'] = final_model_high.predict(test[features])
test['price_p95'] = np.maximum(test['price_p95'], test['price_p05'] + 0.001)

# 7. СОХРАНЕНИЕ SUBMISSION
submission = test[['row_id', 'price_p05', 'price_p95']].sort_values('row_id')
submission.to_csv('submission.csv', index=False)
print("Файл submission.csv успешно сохранен!")

Размер обучающей выборки: (23176, 25)
Размер валидационной выборки: (5820, 25)
Обучение моделей с использованием валидации и Early Stopping...
0:	learn: 0.1464587	test: 0.1464922	best: 0.1464922 (0)	total: 170ms	remaining: 5m 38s
100:	learn: 0.0974140	test: 0.1164734	best: 0.1157368 (81)	total: 3.62s	remaining: 1m 8s
200:	learn: 0.0925590	test: 0.1429650	best: 0.1157368 (81)	total: 7.64s	remaining: 1m 8s
300:	learn: 0.0895062	test: 0.1562941	best: 0.1157368 (81)	total: 11.4s	remaining: 1m 4s
400:	learn: 0.0868456	test: 0.1615190	best: 0.1157368 (81)	total: 15.2s	remaining: 1m
500:	learn: 0.0844409	test: 0.1615846	best: 0.1157368 (81)	total: 19.1s	remaining: 57s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.1157367778
bestIteration = 81

Shrink model to first 82 iterations.
0:	learn: 0.0982666	test: 0.0982913	best: 0.0982913 (0)	total: 33.4ms	remaining: 1m 6s
100:	learn: 0.0834741	test: 0.0942800	best: 0.0941261 (64)	total: 3.76s	remaining: 1m 10s
200:	learn: 0.08